In [3]:
import re
from collections import Counter

import os.path
import matplotlib.pyplot as plt
import operator
import pandas as pd
import praw
import os as os
import pickle 
pd.options.display.max_columns = 999

from os import listdir
from os.path import isfile, join

import math

#Visualization imports
from bokeh.plotting import figure, show
from bokeh.core.properties import value
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import show, output_notebook
output_notebook()
import numpy as np
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge

#Keywords that we chose to anaylize
keys = ['past', 'present', 'future', 'swear', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'discrep']


Loading BokehJS ...

# PLOTS

In [4]:
# Plot the monthly sentiments of the given articles to compare different keywords in a given sentiment
color =["#e6194b","#3cb44b","#ffe119","#0082c8","#f58231","#911eb4","#46f0f0","#f032e6","#d2f53c","#fabebe","#008080","#e6beff","#aa6e28","#fffac8","#800000","#aaffc3","#808000","#ffd8b1","#000080","#808080","#FFFFFF","#000000","#1F77B4", "#B85A0D", '#878787', "#E377C2", '#66ccff', '#cc0066', '#cccc00', '#98DF8A', "#82853B","#92a8d1","#034f84","#f7cac9","#f7786b","#d5f4e6","#80ced6","#fefbd8","#618685","#ffef96","#50394c","#b2b2b2","#f4e1d2","#deeaee","#b1cbbb","#eea29a","#c94c4c","#3e4444","#82b74b","#405d27","#c1946a","#b9936c","#dac292",
"#e6e2d3","#c4b7a6","#6b5b95","#feb236","#d64161","#ff7b25","#eaece5","#b2c2bf","#c0ded9","#3b3a30","#c8c3cc","#563f46","#8ca3a3","#484f4f","#686256",
"#c1502e","#587e76","#a96e5b","#bccad6","#8d9db6","#667292","#f1e3dd","#cfe0e8","#b7d7e8","#87bdd8","#daebe8","#fbefcc","#f9ccac","#f4a688","#e0876a",
"#f9d5e5","#eeac99","#e06377","#c83349","#5b9aa0","#d6d4e0","#b8a9c9","#622569","#96ceb4","#ffeead","#ffcc5c","#ff6f69","#588c7e","#f2e394","#f2ae72","#d96459"]*5

def plot_monthly_sentiments(keywords,log=False, sentiment='num_comments'):
    df = pd.read_csv('data/sentiment_analysis_data.csv')
    df.drop(['RC_2017-01.csv', 'RC_2017-02.csv', 'RC_2017-03.csv'], axis=1, inplace=True)
    axis_type = 'linear'
    if (log):
        axis_type = 'log'
    df1 = pd.DataFrame(columns=df.columns)
    for keyword in keywords:
        df1 = pd.concat([df1, df[(df.keyword==keyword) & (df.sentiment==sentiment)]])
    df=df1.copy()
    for col in df.columns[2:]:
        df.rename(columns={col:col[0:-4]}, inplace = True)

    data = {'keywords': list(df.columns[2:])}
    for i, row in df.iterrows():
        data[row.keyword] = list(row.values[2:])
    source = ColumnDataSource(data=data)

    p = figure(x_range=list(df.columns[2:]), plot_width = 970, title= sentiment,  
               y_axis_type=axis_type)
    i=0
    for indx, row in df.iterrows():
        p.line(data['keywords'], data[row.keyword], color=color[i], legend=value(row.keyword), line_width=1.5)
        i+=1
    p.xaxis.major_label_orientation = math.pi/3
    p.grid.grid_line_alpha=1
    p.x_range.range_padding = 0.01
    p.legend.location = "top_left"
    p.legend.click_policy="hide"
    
    tab = Panel(child=p, title=sentiment)
    return tab

def plot_monthly_multiple_sentiments(keywords, log=False):
    tabs=[]
    sentiments = ['num_comments', 'num_deleted', 'num_removed', 'past', 'present', 'future', 'swear', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'discrep', 'comments_analyzed']
    for sentiment in sentiments:
        tab=plot_monthly_sentiments(keywords, log=log, sentiment=sentiment)
        tabs.append(tab)
    tabs = Tabs(tabs=tabs)
    show(tabs)


Plot the Sentiment analysis of the given articles for all the months together in a multibar visualization

In [5]:


color =["#e6194b","#3cb44b","#ffe119","#0082c8","#f58231","#911eb4","#46f0f0","#f032e6","#d2f53c","#fabebe","#008080","#e6beff","#aa6e28","#fffac8","#800000","#aaffc3","#808000","#ffd8b1","#000080","#808080","#FFFFFF","#000000","#1F77B4", "#B85A0D", '#878787', "#E377C2", '#66ccff', '#cc0066', '#cccc00', '#98DF8A', "#82853B","#92a8d1","#034f84","#f7cac9","#f7786b","#d5f4e6","#80ced6","#fefbd8","#618685","#ffef96","#50394c","#b2b2b2","#f4e1d2","#deeaee","#b1cbbb","#eea29a","#c94c4c","#3e4444","#82b74b","#405d27","#c1946a","#b9936c","#dac292",
"#e6e2d3","#c4b7a6","#6b5b95","#feb236","#d64161","#ff7b25","#eaece5","#b2c2bf","#c0ded9","#3b3a30","#c8c3cc","#563f46","#8ca3a3","#484f4f","#686256",
"#c1502e","#587e76","#a96e5b","#bccad6","#8d9db6","#667292","#f1e3dd","#cfe0e8","#b7d7e8","#87bdd8","#daebe8","#fbefcc","#f9ccac","#f4a688","#e0876a",
"#f9d5e5","#eeac99","#e06377","#c83349","#5b9aa0","#d6d4e0","#b8a9c9","#622569","#96ceb4","#ffeead","#ffcc5c","#ff6f69","#588c7e","#f2e394","#f2ae72","#d96459"]*5

def plot_multibar_sentiment(articles):
    keys = ['past', 'present', 'future', 'swear', 'affect', 'positive', 'negative', 'anxiety', 'anger', 'sadness', 'discrepancy']
    
    df = pd.read_csv('data/summarized_sentiments.csv', index_col=0)
    df.rename(index=str, columns={"posemo": "positive", "negemo": "negative", "anx":"anxiety", "sad":"sadness", "discrep": "discrepancy"}, inplace=True)
    with open('data/list_num_comments_analyzed', 'rb') as fp:
        dict_num_posts = pickle.load(fp)
    art = articles.copy()
    for article in art: 
        #We only plot the article with more than 50 comments
        if (dict_num_posts[article]<50):
            articles.remove(article)
    data = {'LIWC' : keys}
    for i, article in enumerate(articles):
        data[article] = list(df.loc[article])
    for art in articles:
        print('Number of posts '+ art +': '+ str(dict_num_posts[art]))
    source = ColumnDataSource(data=data)
    p = figure(x_range=keys, plot_height=500, plot_width = 970, title="Sentiment Analysis",
               toolbar_location=None, tools="")

    incr=0
    base = 0.8/len(articles)
    for i,art in enumerate(articles):
        p.vbar(x=dodge('LIWC', -base+incr, range=p.x_range), top=art, width=base-0.05, source=source,
           color=color[i], legend=value(art))
        incr+=base

    #p.vbar(x=dodge('Topics',  0.25, range=p.x_range), top='2017', width=0.2, source=source,
           #color="#e84d60", legend=value("2017"))

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_right"
    p.legend.click_policy="hide"

    show(p)

## Main Visualization: Sentiment Analysis for different articles

Visualization of the values of the sentiment analysis of the given articles.

In [8]:
pd.read_csv('data/summarized_sentiments.csv', index_col=0).head()

,past,present,future,swear,affect,posemo,negemo,anx,anger,sad,discrep
human-computer interaction,0.104476,0.069073,0.013627,0.011799,0.068891,0.065284,0.015819,0.005227,0.013402,0.002917,0.064798
open platform,0.025649,0.137604,0.019203,0.008909,0.068433,0.051659,0.025525,0.005797,0.025469,0.006276,0.025265
open-source software,0.027010,0.087632,0.013230,0.002933,0.059512,0.048287,0.018546,0.003263,0.008498,0.003537,0.024404
temporary work,0.023310,0.081260,0.010680,0.005202,0.046297,0.030836,0.016434,0.002500,0.009245,0.002658,0.020701
digital identity,0.016820,0.099356,0.012082,0.003806,0.068799,0.055561,0.017415,0.005032,0.010696,0.003135,0.022286


In [6]:
#Select the articles and the topic (social or technological) to visualize.
articles = ['gdpr', 'fake news', 'intellectual property']
plot_multibar_sentiment(articles)

Number of posts gdpr: 35332.0
Number of posts fake news: 510531.0
Number of posts intellectual property: 21335.0


In [9]:
pd.read_csv('data/sentiment_analysis_data.csv')

,sentiment,keyword,RC_2017-01.csv,RC_2017-02.csv,RC_2017-03.csv,RC_2017-04.csv,RC_2017-05.csv,RC_2017-06.csv,RC_2017-07.csv,RC_2017-08.csv,RC_2017-09.csv,RC_2017-10.csv,RC_2017-11.csv,RC_2017-12.csv,RC_2018-01.csv,RC_2018-02.csv,RC_2018-03.csv,RC_2018-04.csv,RC_2018-05.csv,RC_2018-06.csv,RC_2018-07.csv,RC_2018-08.csv
0,num_comments,human-computer interaction,2.0,2.0,0.0,5.0,0.0,6.0,0.0,4.0,5.0,1.0,0.0,40.0,7.0,5.0,9.0,12.0,3.0,15.0,1.0,0.0
1,num_comments,open platform,92.0,3.0,0.0,18.0,20.0,6.0,1.0,13.0,7.0,1.0,116.0,8.0,61.0,174.0,117.0,48.0,73.0,172.0,59.0,126.0
2,num_comments,open-source software,9.0,31.0,0.0,254.0,93.0,16.0,5.0,148.0,131.0,23.0,1.0,18.0,37.0,20.0,65.0,382.0,76.0,90.0,13.0,48.0
3,num_comments,digital commons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,num_comments,temporary work,40.0,124.0,3.0,114.0,31.0,38.0,61.0,91.0,75.0,202.0,108.0,108.0,118.0,49.0,80.0,214.0,71.0,33.0,305.0,50.0
5,num_comments,digital identity,2.0,33.0,0.0,5.0,55.0,30.0,46.0,1.0,21.0,23.0,59.0,12.0,19.0,63.0,38.0,308.0,6.0,21.0,15.0,45.0
6,num_comments,smart city,33.0,59.0,0.0,73.0,33.0,129.0,97.0,40.0,102.0,150.0,4793.0,123.0,84.0,104.0,66.0,102.0,79.0,119.0,24.0,67.0
7,num_comments,robot tax,22.0,126.0,0.0,230.0,90.0,28.0,3.0,99.0,338.0,1018.0,0.0,64.0,14.0,7.0,12.0,11.0,1.0,0.0,2.0,0.0
8,num_comments,blockchain,3501.0,3867.0,84.0,3795.0,5089.0,8199.0,7698.0,11017.0,10176.0,15551.0,13150.0,23515.0,29859.0,27100.0,34645.0,37744.0,38446.0,40684.0,28432.0,29484.0
9,num_comments,sustainability,299.0,351.0,4.0,592.0,686.0,4673.0,611.0,400.0,285.0,617.0,609.0,574.0,620.0,602.0,527.0,1000.0,1416.0,718.0,721.0,2240.0


In [10]:
plot_monthly_multiple_sentiments(['twitter', 'google', 'facebook', 'amazon', 'zte', 'tencent', 'internet'])